In [1]:
import dxdata
import pandas as pd
import os

# Initialize dxdata engine
engine = dxdata.connect(dialect="hive+pyspark")
project = os.popen("dx env | grep project- | awk -F '\t' '{print $2}'").read().rstrip()
record = os.popen("dx describe *dataset | grep  record- | awk -F ' ' '{print $2}'").read().rstrip().split('\n')[0]
DATASET_ID = project + ":" + record
dataset = dxdata.load_dataset(id=DATASET_ID)
data = dataset['participant']
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("YourAppName")\
    .config("spark.kryoserializer.buffer.mb", "24")\
    .config("spark.kryoserializer.buffer.max.mb", "2000")\
    .getOrCreate()



# Find by field name
field_eid = pheno.find_field(name="eid")

# Find by exact title
field_sex = pheno.find_field(title="Sex")
field_age = pheno.find_field(title="Age at recruitment")
field_height = pheno.find_field(title="Standing height | Instance 0")

In [4]:
field_list = [field_eid, field_height, field_sex, field_age]
field_list

[<Field "eid">, <Field "p50_i0">, <Field "p31">, <Field "p21022">]

## prepare recorders 

In [16]:
df_all_fields = pd.DataFrame(columns = ['entity','field_name','filed_title'])
for entity in dataset.entities:
    entity_name  = entity.name
    
    for field in entity.fields:
        field_name = field.name
        filed_title = field.title
        df_all_fields.loc[len(df_all_fields),] = [entity_name,field_name,filed_title]
df_all_fields['downloaded']=[False]*len(df_all_fields)
df_all_fields.to_csv('recorder_all_fields.csv',index=False)


In [24]:
for entity in df_all_fields['entity'].unique():
    df_target_entity = df_all_fields.loc[df_all_fields['entity']==entity,]
    df_target_entity.to_csv(f'recorder_{entity}.csv')

# data downloading codes

In [2]:
import warnings 
warnings.filterwarnings('ignore')

In [3]:
fail_list = [52,414]

In [4]:
ind = 815
entity='participant'

interval = 500 # how many files to be downloaded in one go
bulk_num = 30 # how many fields in one file

In [ ]:
from datetime import datetime
start_time = datetime.now()
df_target_entity = pd.read_csv(f'recorder_{entity}.csv')
end = ind+interval

while ind < end:
    try:
        print(f"now download bulk {ind}, from {ind*bulk_num+1} to {ind*bulk_num+bulk_num}")
        field_names = df_target_entity.loc[ind*bulk_num+1:ind*bulk_num+bulk_num,'field_name']
        index = field_names.index
        field_names = ['eid']+list(field_names)
        field_lst = [data.find_field(x) for x in field_names]

        df_download = data.retrieve_fields(engine=engine, fields=field_lst, coding_values="replace")
        df_download = df_download.toPandas()

        df_download.to_csv(f'data/{entity}/{ind}.csv',index=False)

        df_target_entity.loc[index, 'downloaded']=[True]*len(index)
        df_target_entity.loc[index, 'ind']=[ind]*len(index)
        df_target_entity.to_csv(f'recorder_{entity}.csv')
    except Exception as e :
        print(f"error in bulk {ind}, {e}")
        with open('fail_ind_lst.txt','a') as f:
            f.write(f',{ind}')
        f.close()
        fail_list+=[ind]
    ind+=1
print(f'starttime {start_time}, endtime {datetime.now()}')
print(f'average ({datetime.now()-start_time})/interval per file')

now download bulk 815, from 24451 to 24480
now download bulk 816, from 24481 to 24510
now download bulk 817, from 24511 to 24540
now download bulk 818, from 24541 to 24570
now download bulk 819, from 24571 to 24600
now download bulk 820, from 24601 to 24630
now download bulk 821, from 24631 to 24660
now download bulk 822, from 24661 to 24690
now download bulk 823, from 24691 to 24720
now download bulk 824, from 24721 to 24750
now download bulk 825, from 24751 to 24780
now download bulk 826, from 24781 to 24810
now download bulk 827, from 24811 to 24840
now download bulk 828, from 24841 to 24870
now download bulk 829, from 24871 to 24900
now download bulk 830, from 24901 to 24930
now download bulk 831, from 24931 to 24960
now download bulk 832, from 24961 to 24990
now download bulk 833, from 24991 to 25020
now download bulk 834, from 25021 to 25050
now download bulk 835, from 25051 to 25080
now download bulk 836, from 25081 to 25110
now download bulk 837, from 25111 to 25140
now downloa

## segregation of file 52 

In [11]:
from datetime import datetime
start_time = datetime.now()
df_target_entity = pd.read_csv(f'recorder/recorder_{entity}.csv')

ind = 5204
try:
    #field_names = df_target_entity.loc[1561:1570,'field_name'] #5200 passed 
    #field_names = df_target_entity.loc[1571:1571,'field_name']#5202 passed
    #field_names = df_target_entity.loc[1572:1572,'field_name']#5203  
    field_names = df_target_entity.loc[1573:1575,'field_name'] #5204 passed 
    # field_names = df_target_entity.loc[1575:1590,'field_name'] #5205 passed 
    index = field_names.index
    field_lst = [data.find_field(x) for x in list(field_names)]

    df_download = data.retrieve_fields(engine=engine, fields=field_lst, coding_values="replace")
    df_download = df_download.toPandas()

    df_download.to_csv(f'data/{entity}/{ind}.csv',index=False)

    #df_target_entity.loc[index, 'downloaded']=[True]*len(index)
    #df_target_entity.loc[index, 'ind']=[ind]*len(index)
    #df_target_entity.to_csv(f'recorder/recorder_{entity}.csv')
    # fail_list.remove(ind)
except Exception as e:
    print(e)
    print(f"error in bulk {ind}")


print(f' starttime {start_time}, endtime {datetime.now()}')

 starttime 2024-02-27 14:52:42.425171, endtime 2024-02-27 14:53:30.363965


ind = 0
interval = 5

filenames = [f'data/{entity}/{x}.csv' for x in range(ind,ind+interval)]
df_compact = pd.concat(map(pd.read_csv, filenames))
df_compact.to_pickle(f'data/{entity}/{ind}-{interval-1}.pkl')
for file in filenames:
    os.remove(file)


## only download diseases 

In [ ]:
# all_icd 
field_names = ['Diagnoses - ICD10','Date of first in-patient diagnosis - ICD10','Diagnoses - main ICD10','Date of first in-patient diagnosis - main ICD10','Diagnoses - main ICD9']
field_ids = ['eid','p41270']+[f'p41280_a{x}' for x in range(0,259)]
field_lst = [data.find_field(x) for x in list(field_ids)]
df_download = data.retrieve_fields(engine=engine, fields=field_lst, coding_values="replace")
df_download = df_download.toPandas()
df_download.to_csv(f'diseases_all_icd.csv',index=False)

In [ ]:
# main_icd 
field_names = ['Diagnoses - ICD10','Date of first in-patient diagnosis - ICD10','Diagnoses - main ICD10','Date of first in-patient diagnosis - main ICD10','Diagnoses - main ICD9']
field_ids = ['eid','p41202']+[f'p41262_a{x}' for x in range(0,80)]
field_lst = [data.find_field(x) for x in list(field_ids)]
df_download = data.retrieve_fields(engine=engine, fields=field_lst, coding_values="replace")
df_download = df_download.toPandas()
df_download.to_csv(f'diseases_main_icd.csv',index=False)

In [ ]:

# second_icd 
field_names = ['Diagnoses - ICD10','Date of first in-patient diagnosis - ICD10','Diagnoses - main ICD10','Date of first in-patient diagnosis - main ICD10','Diagnoses - main ICD9']
field_ids = ['eid','p41203']
field_lst = [data.find_field(x) for x in list(field_ids)]
df_download = data.retrieve_fields(engine=engine, fields=field_lst, coding_values="replace")
df_download = df_download.toPandas()
df_download.to_csv(f'diseases_second_icd.csv',index=False)
